In [2]:
# Importing the python image library to load in .tiffs
from PIL import Image
import numpy as np
import os

import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [3]:
# LOADING THE TIFFS
# Load in a single image
singleImage = Image.open('/Users/jonathanlindenmann/jonspace/000_aut_23/ESS469/Git/MlGeo_Aikin_project/Aikin_Data/RawMicroprobeMaps/78.7-10-1_Hot/RAW/UGG-W3-87.7-10.1-Full_Al Ka_Al Ka EDS.Tiff')
# Turn this image into an array
singleImageArray = np.asarray(singleImage)
print(singleImageArray.shape)



(695, 1152)


## Break down of the code provided by Akshay:

We have four thin sections, and each one has ten .tiffs in a 'RAW' folder. We want to create a stack of all ten .tiffs for each thin section in an array, then make a list of all four image stacks. 

1. Create the list `thinSections`. This is a `dict` object that contains the thin section `name` and `stack` (the root folder )
2. Create a loop that iterates over every directory in the parent folder with the name 'RAW' and extracts the .tiffs within into a 3D array, then adds the image stack to the `thinSections` list object:
    2.1 Navigates to a directory called 'RAW'
    
    2.1.1 In the 'RAW' directory, use the name of the folder the 'RAW' folder is in as the `thinSectionName` and adds it to the thinSections object as a key:`name` `dict` object.
        
    2.1.2 Find the number of files in the 'RAW' folder. 
        
    2.1.3 For all 'RAW' folders that have files in them, 
            
    2.1.3.1 Create boolean that logs whether or not the image stack is complete. 
    
    2.1.3.2 Create an empty array for the image stack. 
    
    2.1.3.3 Count the number of image stacks/RAW folders being iterated over.
    
    2.1.4 For each .tiff, open the file, turn it into an array, and add it to the `tsArray`, which is the loop's working image stack, at the layer specified by section count. The section count indexes each 2D layer by counting the layers.
            
    Refer to the code to understand the rest. 
            

        

In [4]:
# We know that we can turn one image into an array...can we stack all the images for a thin section into one, 3D array?

# Create an array that will contain each thin section

thinSections = [] #empty LIST for the all stacked thin section arrays

iteration = 0

# Okay, for every folder in our parent folder

for root, dirs, files in os.walk('.//'):
    # './/' means the CURRENT working directory. In this case, it is the one that the jupyter notebook is stored in (perfect!)
       
        # Does the directory contain the word RAW?
        if "RAW" in root:
            # If so, what is the name of the thin section?
            thinSectionName = os.path.basename(os.path.dirname(root))
            thinSections.append(dict({'name': thinSectionName}))
            numberOfFiles = len(files)
            if numberOfFiles > 0:
                arraySet = False #
                tsArray = [] #the array of stacked thin sections
                sectionCount = 0
                for file in files:
                    tsImage = Image.open(os.path.join(root, file))
                    singleImageArray = np.asarray(tsImage)
                    if not arraySet:
                        tsShape = singleImageArray.shape
                        tsArray = np.empty([tsShape[0], tsShape[1], numberOfFiles])
                        arraySet = True
                    tsArray[:, :, sectionCount] = singleImageArray
                    sectionCount = sectionCount + 1
                thinSections[iteration]['stack'] = tsArray # add the array as the 'stack' key's value to the thinSections list
                                                           # to the list item corresponding with the iteration (thin section 1, 2, 3, or ,4.)
            iteration = iteration + 1
            #for file in files:
              #print(os.path.join(root, file)) #prints the filepath (each iteration, so each all .tiffs from all thin sections)


     #this is an array that was just used in the loop to add thin section image stacks to the thinSections list. 
                            #It now contains the image stack from the last thin section. 
print(thinSectionName)
#print("tsArray.shape:", tsArray.shape)
#print ("iteration", iteration)
#print("sectionCount:", sectionCount)
print("thinSections:", thinSections)
#print("length of thinSections:", len(thinSections))
            



78.7-10-2_Hot
thinSections: [{'name': '84.7-NA-2-1_Cold', 'stack': array([[[ 44.        ,  23.        ,  62.        , ...,  10.        ,
         114.        , 147.        ],
        [ 40.        ,  23.        ,  60.        , ...,  10.        ,
         447.        , 148.        ],
        [ 48.        ,  26.99999809,  64.        , ...,  11.        ,
         391.        , 146.        ],
        ...,
        [ 26.        ,  18.        ,  58.        , ...,  14.00000095,
         175.        , 160.        ],
        [ 25.        ,  20.        ,  76.        , ...,   5.        ,
         175.        , 165.        ],
        [ 25.        ,  20.        ,  66.        , ...,   9.        ,
         142.        , 182.        ]],

       [[ 72.        ,  16.        ,  69.        , ...,   9.        ,
         510.        , 142.        ],
        [ 47.        ,  21.        ,  58.        , ...,   8.        ,
         884.        , 127.        ],
        [ 97.        ,  20.        ,  81.        , ...

We now have `thinSections`, which is a list that contains 4 elements. Each element is a thin section, and contains `name: (name of thin section from root dir)` and `stack: (image stack of all the .tiffs of that thin section)`, in other words, it contains a string, and a 3D array: ten layers of 2D arrays.

Next we want to create a widget to work with these image stacks. 

In [7]:
thisThinSection = thinSections[0]
stack =  thisThinSection['stack']
stackShape = stack.shape

# Create a drop down to pick the right thin section
numberofthinsections = len(thinSections)

channel0Dropdown = widgets.Dropdown(
    options=range(numberofthinsections),
    description='Thin section'
)

# Create dropdown widgets for the three dimensions
channel1Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 1:'
)

channel2Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 2:'
)

channel3Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 3:'
)

# Create a button widget to trigger visualization
visualizeBtn = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to visualize',
    icon='check'
)

# Create an output widget to display the image
output = widgets.Output()

def visualize(b):

    recastThinSection = np.empty([stackShape[0], stackShape[1], 3], dtype='int')

    recastThinSection[:,:,0] = stack[:,:,channel1Dropdown.value]
    recastThinSection[:,:,1] = stack[:,:,channel2Dropdown.value]
    recastThinSection[:,:,2] = stack[:,:,channel3Dropdown.value]

    print(recastThinSection)

    # Display the image using Matplotlib
    with output:
        output.clear_output()
        plt.figure(figsize=(8, 6))
        plt.imshow(recastThinSection)  # You can change the colormap as needed
        plt.show()

# Connect the button's click event to the visualization function
visualizeBtn.on_click(visualize)

# Display the widgets
display(channel0Dropdown, channel1Dropdown, channel2Dropdown, channel3Dropdown, visualizeBtn)
display(output)





Dropdown(description='Thin section', options=(0, 1, 2, 3), value=0)

Dropdown(description='Channel 1:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

Dropdown(description='Channel 2:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

Dropdown(description='Channel 3:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0)

Button(button_style='success', description='Visualize', icon='check', style=ButtonStyle(), tooltip='Click to v…

Output()

[[[44 44 44]
  [40 40 40]
  [48 48 48]
  ...
  [26 26 26]
  [25 25 25]
  [25 25 25]]

 [[72 72 72]
  [47 47 47]
  [97 97 97]
  ...
  [29 29 29]
  [31 31 31]
  [43 43 43]]

 [[76 76 76]
  [78 78 78]
  [75 75 75]
  ...
  [32 32 32]
  [45 45 45]
  [57 57 57]]

 ...

 [[28 28 28]
  [20 20 20]
  [36 36 36]
  ...
  [17 17 17]
  [17 17 17]
  [ 8  8  8]]

 [[25 25 25]
  [16 16 16]
  [35 35 35]
  ...
  [ 9  9  9]
  [19 19 19]
  [18 18 18]]

 [[26 26 26]
  [27 27 27]
  [25 25 25]
  ...
  [12 12 12]
  [25 25 25]
  [23 23 23]]]


[[[44 44 44]
  [40 40 40]
  [48 48 48]
  ...
  [26 26 26]
  [25 25 25]
  [25 25 25]]

 [[72 72 72]
  [47 47 47]
  [97 97 97]
  ...
  [29 29 29]
  [31 31 31]
  [43 43 43]]

 [[76 76 76]
  [78 78 78]
  [75 75 75]
  ...
  [32 32 32]
  [45 45 45]
  [57 57 57]]

 ...

 [[28 28 28]
  [20 20 20]
  [36 36 36]
  ...
  [17 17 17]
  [17 17 17]
  [ 8  8  8]]

 [[25 25 25]
  [16 16 16]
  [35 35 35]
  ...
  [ 9  9  9]
  [19 19 19]
  [18 18 18]]

 [[26 26 26]
  [27 27 27]
  [25 25 25]
  ...
  [12 12 12]
  [25 25 25]
  [23 23 23]]]


[[[44 44 44]
  [40 40 40]
  [48 48 48]
  ...
  [26 26 26]
  [25 25 25]
  [25 25 25]]

 [[72 72 72]
  [47 47 47]
  [97 97 97]
  ...
  [29 29 29]
  [31 31 31]
  [43 43 43]]

 [[76 76 76]
  [78 78 78]
  [75 75 75]
  ...
  [32 32 32]
  [45 45 45]
  [57 57 57]]

 ...

 [[28 28 28]
  [20 20 20]
  [36 36 36]
  ...
  [17 17 17]
  [17 17 17]
  [ 8  8  8]]

 [[25 25 25]
  [16 16 16]
  [35 35 35]
  ...
  [ 9  9  9]
  [19 19 19]
  [18 18 18]]

 [[26 26 26]
  [27 27 27]
  [25 25 25]
  ...
  [12 12 12]
  [25 25 25]
  [23 23 23]]]


In [ ]:
Add the names of each layer to the dropdown lists for each channel.